In [1]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re

In [13]:
# function to grab all HTML content from a page, returns a BeautifulSoup object
def get_page_soup(url, wait_string): 

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome()

    # Navigate to the webpage
    driver.get(url)

    if "mostplayed" in url:
        # Wait up to 5 seconds for an element to be present on the page
        wait = WebDriverWait(driver, 5)
        _    = wait.until(lambda driver: driver.find_element(By.XPATH, wait_string))

    try:

        # wait for an age gate to potentially appear
        wait = WebDriverWait(driver, 5)
        _    = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "age_gate")))

        # test if the page contains an age gate
        # driver.find_element(By.XPATH, "//div[contains(@class, 'age_gate')]")

        # click the ageYear option for 1950
        driver.find_element(By.XPATH, "//select[contains(@id, 'ageYear')]/option[text()='1950']").click()

        # click the "View Page" button
        driver.find_element(By.XPATH, "//a[contains(@id, 'view_product_page_btn')]").click()

        # wait for the page to load
        # wait = WebDriverWait(driver, 5)
        # _    = wait.until(lambda driver: driver.find_element(By.XPATH, wait_string))

    except: pass

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # close the browser
    driver.quit()

    # return the soup
    return soup

In [3]:
# Get top 100 games from Steam
steam_soup = get_page_soup(url='https://store.steampowered.com/charts/mostplayed', wait_string="//tr[contains(@class, '_2-')]")

In [14]:
# get all the games listed
games = steam_soup.find_all('tr', class_=lambda value: value and value.startswith('_2-'))

# create a dictionary to store the data for each game from this page
game_data = {f'{int(game.find_all("td")[1].text):03d}': {'name' : game.find('a').text   ,
                                                    'link' : game.find('a')['href'],
                                                    'soup' : get_page_soup(url=game.find('a')['href'], wait_string="//div[contains(@class, 'game_area_sys_req_full')]"),} for game in games}

df = pd.DataFrame(game_data).T
df

,name,link,soup
001,Counter-Strike 2,https://store.steampowered.com/app/730/Counter...,"[[[\n, <meta content=""text/html; charset=utf-8..."
002,Dota 2,https://store.steampowered.com/app/570/Dota_2?...,"[[[\n, <meta content=""text/html; charset=utf-8..."
003,HELLDIVERS™ 2,https://store.steampowered.com/app/553850/HELL...,"[[[\n, <meta content=""text/html; charset=utf-8..."
004,PUBG: BATTLEGROUNDS,https://store.steampowered.com/app/578080/PUBG...,"[[[\n, <meta content=""text/html; charset=utf-8..."
005,Baldur's Gate 3,https://store.steampowered.com/app/1086940/Bal...,"[[[\n, <meta content=""text/html; charset=utf-8..."
...,...,...,...
096,Squad,https://store.steampowered.com/app/393380/Squa...,"[[[\n, <meta content=""text/html; charset=utf-8..."
097,Sea of Thieves 2023 Edition,https://store.steampowered.com/app/1172620/Sea...,"[[[\n, <meta content=""text/html; charset=utf-8..."
098,New World,https://store.steampowered.com/app/1063730/New...,"[[[\n, <meta content=""text/html; charset=utf-8..."
099,Satisfactory,https://store.steampowered.com/app/526870/Sati...,"[[[\n, <meta content=""text/html; charset=utf-8..."


In [35]:
game_data['001']['soup'].find_all('div', class_='game_area_sys_req_full')[0].find('strong').text.lower()
print(game_data['001']['soup'].select_one('li:contains("OS")').text.replace('OS:', '').strip())
print(game_data['001']['soup'].select_one('li:contains("Processor")').text.replace('Processor:', '').strip())
print(game_data['001']['soup'].select_one('li:contains("Memory")').text.replace('Memory:', '').strip())
print(game_data['001']['soup'].select_one('li:contains("Graphics")').text.replace('Graphics:', '').strip())
print(game_data['001']['soup'].select_one('li:contains("DirectX")').text.replace('DirectX:', '').strip())
print(game_data['001']['soup'].select_one('li:contains("Storage")').text.replace('Storage:', '').strip())

Windows® 10
4 hardware CPU threads - Intel® Core™ i5 750 or higher
8 GB RAM
Video card must be 1 GB or more and should be a DirectX 11-compatible with support for Shader Model 5.0
Graphics: Video card must be 1 GB or more and should be a DirectX 11-compatible with support for Shader Model 5.0
85 GB available space


In [ ]:
# Loop over all games again to get the soup